In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import time
%load_ext autoreload
%autoreload 2

In [2]:
from proj1_helpers import *
DATA_TRAIN_PATH = 'train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)


In [3]:
def split_data(x, y, ratio,seed=1):
    """split the dataset based on the split ratio."""
    # set seed
    np.random.seed(seed)
    # ***************************************************
    # INSERT YOUR CODE HERE
    # split the data based on the given ratio: TODO    
    c = list(zip(x, y))

    np.random.shuffle(c)

    x, y = zip(*c)
    
    ind = round(ratio * len(x))
    ind2 = len(x)
    x = np.array(x)
    trainx = x[0:ind,:]
    trainy = y[0:ind]
    testx = x[ind:ind2,:]
    testy = y[ind:ind2]
    
    return trainx, testx,trainy,testy

In [6]:
# Removing bothering data and centering
tX[tX==-999] = 0
m = np.mean(tX, axis=0)
centered_tX = tX - m

centered_tX[centered_tX==0] = float('nan')
stdevtrain = np.nanstd(centered_tX, axis=0);
centered_tX[centered_tX==float('nan')] = 0
standardized_tX = centered_tX / stdevtrain

d = len(standardized_tX[0])
n = len(standardized_tX)
new_dims = 0
    
mat = np.zeros((n, new_dims + int(d * (d + 1) / 2 + d)))
q = 0
tao = d 

# From 3 sec to 0.01
mat[:standardized_tX.shape[0], :standardized_tX.shape[1]] = standardized_tX

start = time.time()

for i in range (n):
    for monsoon in range (d):
        for t in range (monsoon,d): 
            mat[i,tao] = standardized_tX[i,t] * standardized_tX[i,monsoon]
            tao=tao+1
    tao=d
    
end = time.time()
print("It took:", end - start, "to run.")

start = time.time()

indices = 
for i in range (n):
    for monsoon in range (d):
        for t in range (monsoon,d): 
            indices[i,tao] = [[i,t], [i,monsoon]]
            tao=tao+1
    tao=0

end = time.time()
print("It took:", end - start, "to run.")
#for t in range (3,11):
#    for i in range (30):
#        mat=np.column_stack((mat,standardized_tX[:,i]**t))

end = time.time()
print("It took:", end - start, "to run.")
    


It took: 84.6303768157959 to run.


TypeError: list indices must be integers or slices, not tuple

In [4]:
#excluded=[]
#for i in range (len(mat[0])):
#    a=np.corrcoef(y,mat[:,i])
#    if abs(a[0,1])<10**-3:
#        excluded.append(i)

In [5]:
#mat=np.delete(mat,excluded,axis=1)

In [6]:
m2=np.mean(mat,axis=0)
centered_mat=mat-m2
centered_mat[mat==0]=0

centered_mat[centered_mat==0]=float('nan')
stdev=np.nanstd(centered_mat,axis=0);
centered_mat[centered_mat==float('nan')]=0
standardized_mat=centered_mat / stdev

In [7]:
num_samples=len(standardized_mat)
tx = np.c_[np.ones(num_samples), standardized_mat]
#y=list(y)
#y[y==-1]=0

In [8]:
def compute_loss(y, tx, w):
    """Calculate the loss.
    
    You can calculate the loss using mse or mae.
    """
    # ***************************************************
    e=y-np.dot(tx,w)
    mse=np.dot(e.transpose(),e)/(2*len(tx))
    return mse
    # ***************************************************
    raise NotImplementedError

In [9]:
tX,testx,y,testy=split_data(tx, y, 0.8,seed=1)

In [10]:
from __future__ import division
import numpy as np 
def logistic(a):
    return 1.0 / (1 + np.exp(-a))
def irls(X, y):
    theta = np.zeros(X.shape[1])
    theta_ = np.inf
    eps=60000
    lamda=5
    for aqua in range (11):
        grad=np.zeros(X.shape[1])
        a = np.dot(X, theta)
        #a=math.log10(abs(a))*sign(a)
        #print(min(a),max(a))
        pi = logistic(a)        
        SX = X * (pi - pi*pi).reshape(-1,1)
        XSX = np.dot(X.T,SX)
        #+lamda*np.eye((len(X[0])))
        for aw in range (len(X)):
            grad=grad+(-1/len(X))*(y[aw]*X[aw,:]*logistic(-y[aw]*np.dot(X[aw,:],theta)))
        
        
     
        theta = theta-eps*np.linalg.solve(XSX,grad)
        print(sum(y==np.sign(np.dot(X,theta)))/len(y))
       
        if aqua%5==0 and aqua!=0:
            eps=eps*0.8
    return theta

In [14]:
theta=irls(tX,y)

0.82505
0.826655
0.828295
0.829095
0.831
0.831925
0.832935
0.833575


/home/romain/.local/lib/python3.5/site-packages/ipykernel/__main__.py:4: RuntimeWarning: overflow encountered in exp


0.834155
0.83477
0.834845


In [15]:
DATA_TEST_PATH = 'test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [16]:
weights=theta
testx=tX_test
#testx=np.delete(testx,[14,15,17,18,24,25,27,28],axis=1)
testx[testx==-999]=0
#m2=np.mean(testx,axis=0)
centered_testx=testx-m
centered_testx[testx==-999]=0
#standardized_testx=centered_testx / np.std(centered_testx, axis=0)
#centered_testx[centered_testx==0]=float('nan')
#stdevtest=np.nanstd(centered_testx,axis=0);
#centered_testx[centered_testx==float('nan')]=0
standardized_testx=centered_testx / stdevtrain

d=len(standardized_testx[0])
n=len(standardized_testx)
new_dims=0
#for t in range (1,d+1):
#    new_dims=new_dims+t*(t+1)/2
mat=np.zeros((n,new_dims+int(d*(d+1)/2+d)))
q=0
tao=d 

for i in range (n):
    for t in range (d):  
        mat[i,t]=standardized_testx[i,t]

        
for i in range (n):
    for mk in range (d):
        for t in range (mk,d): 
            mat[i,tao]=standardized_testx[i,t]*standardized_testx[i,q]
            tao=tao+1
        q=q+1    
    tao=d  
    q=0
    
    
for t in range (3,11):
    for i in range (30):
        mat=np.column_stack((mat,standardized_testx[:,i]**t))


centered_mat=mat-m2
centered_mat[mat==0]=0

#centered_mat[centered_mat==0]=float('nan')
#stdev=np.nanstd(centered_mat,axis=0);
#centered_mat[centered_mat==float('nan')]=0
standardized_testmat=centered_mat / stdev

#tao=int(d*(d+1)/2+d)
    
#for i in range (n):
#    for r in range (d):
#        for monsoon in range (r,d):
#            for t in range (monsoon,d): 
#                mat[i,tao]=standardized_tX[i,t]*standardized_tX[i,monsoon]*standardized_tX[i,r]
#                tao=tao+1
#    tao=int(d*(d+1)/2+d)  
    
num_samples=len(standardized_testmat)
final_testx = np.c_[np.ones(num_samples), standardized_testmat]
tX_test=final_testx

MemoryError: 

In [13]:
OUTPUT_PATH = '/Users/alperkose/Desktop/deneme8.csv' # TODO: fill in desired name of output file for submission
y_pred = -predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)